In [8]:

# from florence import Florence2
# import time
# import matplotlib.pyplot as plt
# from PIL import Image

# florence = Florence2(gpu=True)

In [9]:
# # Initialize Florence2

# # Load original image
# original_image = Image.open("test.jpg")

# # Test text bbox detection
# print("\nTesting text bbox detection...")
# bbox_image = original_image.copy()
# start_time = time.time()
# bbox_result = florence.text_bbox("person", bbox_image, draw=True)['<OPEN_VOCABULARY_DETECTION>']['bboxes']
# bbox_time = time.time() - start_time
# print(f"BBox result: {bbox_result}")
# print(f"bbox time {bbox_time}")

In [10]:
import os
import h5py
import torch
import numpy as np

# def get_norm_stats(dataset_dir, num_episodes):
#     all_qpos_data = []
#     all_action_data = []
#     for episode_idx in range(num_episodes):
#         dataset_path = os.path.join(dataset_dir, f'episode_{episode_idx}.hdf5')
#         with h5py.File(dataset_path, 'r') as root:
#             qpos = root['/observations/qpos'][()]
#             qvel = root['/observations/qvel'][()]
#             action = root['/action'][()]
#         all_qpos_data.append(torch.from_numpy(qpos))
#         all_action_data.append(torch.from_numpy(action))
#     all_qpos_data = torch.stack(all_qpos_data)
#     all_action_data = torch.stack(all_action_data)
#     all_action_data = all_action_data

#     # normalize action data
#     action_mean = all_action_data.mean(dim=[0, 1], keepdim=True)
#     action_std = all_action_data.std(dim=[0, 1], keepdim=True)
#     action_std = torch.clip(action_std, 1e-2, np.inf) # clipping

#     # normalize qpos data
#     qpos_mean = all_qpos_data.mean(dim=[0, 1], keepdim=True)
#     qpos_std = all_qpos_data.std(dim=[0, 1], keepdim=True)
#     qpos_std = torch.clip(qpos_std, 1e-2, np.inf) # clipping

#     stats = {"action_mean": action_mean.numpy().squeeze(), "action_std": action_std.numpy().squeeze(),
#              "qpos_mean": qpos_mean.numpy().squeeze(), "qpos_std": qpos_std.numpy().squeeze(),
#              "example_qpos": qpos}

#     return stats




In [11]:
import open3d as o3d
data= 'data_dir/episode_0.hdf5'
root = h5py.File(data,'r')


pcs = []
for i in range(0,400):
    pc = root[f'/observations/pc/{i}']

    pc_array = np.array(pc, dtype=np.float32)  

    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pc_array)
    pcs.append(pc)

In [12]:
# import open3d as o3d
# import numpy as np

# # Load a point cloud
# pc = pcs[0]  # or any loaded point cloud

# # Your camera extrinsic matrix
# extrinsic = np.array([
#     [-0.05323435, -0.67890574,  0.73229304, -0.57448249],
#     [-0.11560661,  0.73259193,  0.67077878, -0.38695018],
#     [-0.99186754, -0.04894944, -0.11748505,  0.55973048],
#     [0.0,         0.0,         0.0,          1.0]
# ])

# # Create the visualizer and window
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.add_geometry(pc)

# # Get current intrinsics
# view_ctl = vis.get_view_control()
# cam_params = view_ctl.convert_to_pinhole_camera_parameters()

# # Replace the extrinsic
# cam_params.extrinsic = extrinsic
# view_ctl.convert_from_pinhole_camera_parameters(cam_params)

# # Run visualizer
# vis.run()
# vis.destroy_window()


In [13]:
import os
import numpy as np
import open3d as o3d

output_dir = 'new_frames'
os.makedirs(output_dir, exist_ok=True)

extrinsic = np.array([
    [-0.05323435, -0.67890574,  0.73229304, -0.57448249],
    [-0.11560661,  0.73259193,  0.67077878, -0.38695018],
    [-0.99186754, -0.04894944, -0.11748505,  0.55973048],
    [0.0,          0.0,          0.0,         1.0]
])

vis = o3d.visualization.Visualizer()
vis.create_window(visible=False, width=640, height=480)

pc_vis = o3d.geometry.PointCloud()
pc_vis.points = pcs[0].points
if pcs[0].has_colors():
    pc_vis.colors = pcs[0].colors
vis.add_geometry(pc_vis)

view_ctl = vis.get_view_control()
cam_param = view_ctl.convert_to_pinhole_camera_parameters()
cam_param.extrinsic = extrinsic
view_ctl.convert_from_pinhole_camera_parameters(cam_param)

for i, pc_src in enumerate(pcs):
    pc_vis.points = pc_src.points
    if pc_src.has_colors():
        pc_vis.colors = pc_src.colors

    vis.update_geometry(pc_vis)

    # Re-apply extrinsic every frame (important!)
    view_ctl.convert_from_pinhole_camera_parameters(cam_param)

    vis.poll_events()
    vis.update_renderer()

    img = vis.capture_screen_float_buffer(False)
    img = (np.asarray(img) * 255).astype(np.uint8)
    o3d.io.write_image(f'{output_dir}/frame_{i:04d}.png',
                       o3d.geometry.Image(img))

vis.destroy_window()
